In [1]:
from keras.models import Sequential
from keras.layers import Dense
from config import max_letters, language_tags
import numpy as np
from unidecode import unidecode
import pandas as pd

Using TensorFlow backend.


In [2]:
def convert_word_to_vector(lst, max_word_length):
    new_list = []
    for item in lst:
        vec = ''
        n = len(item)
        for x in range(n):
            current_letter = item[x]
            ind = ord(current_letter)-97
            placeholder = (str(0)*ind) + str(1) + str(0)*(25-ind)
            vec = vec + placeholder
        if n < max_word_length:
            excess = max_word_length-n
            vec = vec + str(0)*26*excess
        new_list.append(vec)
    return new_list

In [3]:
network = Sequential()
network.add(Dense(200, input_dim=26*max_letters, activation='sigmoid'))
network.add(Dense(150, activation='sigmoid'))
network.add(Dense(100, activation='sigmoid'))
network.add(Dense(100, activation='sigmoid'))
network.add(Dense(len(language_tags), activation='softmax'))
network.load_weights('weights.hdf5')
network.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def predict_lang_nn(word):
    dic = []
    scores = []
    if len(word) <= max_letters:
        word = word.lower()
        word = unidecode(word)
    dic.append(word)
    vct_str = convert_word_to_vector(dic, max_letters)
    vct = np.zeros((1, 26 * max_letters))
    count = 0
    for digit in vct_str[0]:
        vct[0, count] = int(digit)
        count += 1
    prediction_vct = network.predict(vct)
    langs = list(language_tags.keys())
    for i in range(len(language_tags)):
        lang = langs[i]
        scores.append(prediction_vct[0][i])
    final_out = [0,0]
    for i in range(len(language_tags)):
        if(final_out[0]<scores[i]):
            final_out[0] = scores[i];
            final_out[1] = i
    return langs[final_out[1]]

def predict_lang_uni(word):
    si_count = 0
    en_count = 0
    other_count = 0

    for char in word:
        ord_val = ord(char)
        if (65 <= ord_val <= 90) or (97 <= ord_val <= 122):
            en_count += 1
        elif 3456 <= ord_val <= 3583:
            si_count += 1
        else:
            other_count += 1

    si_presen = si_count / len(word)
    en_presen = en_count / len(word)
    other_presen = other_count / len(word)

    lang_dict = {'si': si_presen, 'en': en_presen, 'other': other_presen}

    return max(lang_dict, key=lang_dict.get)

In [6]:
data = pd.read_excel('test_data_sets/test_set.xlsx', sep=" ")
## data = data.dropna()
# data

In [12]:
## Evaluating NN model
en_en_nn = 0
en_si_nn = 0
si_si_nn = 0
si_en_nn = 0

for i in range(len(data['word'])):
    try:
        if(len(data['word'][i])<=max_letters):
            prediction = predict_lang_nn(data['word'][i])
            if(data['tag'][i]=='en'):
                if(prediction == 'en'):
                    en_en_nn += 1
                elif(prediction == 'si'):
                    en_si_nn += 1
#                     print (data['word'][i])
            elif(data['tag'][i]=='si'):
                if(prediction == 'si'):
                    si_si_nn += 1
                elif(prediction == 'en'):
                    si_en_nn += 1
#                     print (data['word'][i])
    except:
         continue
            
## Evaluating Unicode model
en_en_uni = 0
en_si_uni = 0
si_si_uni = 0
si_en_uni = 0

for i in range(len(data['word'])):
    try:
        if(len(data['word'][i])<=max_letters):
            prediction = predict_lang_uni(data['word'][i])
            if(data['tag'][i]=='en'):
                if(prediction == 'en'):
                    en_en_uni += 1
                elif(prediction == 'si'):
                    en_si_uni += 1
#                     print (data['word'][i])
            elif(data['tag'][i]=='si'):
                if(prediction == 'si'):
                    si_si_uni += 1
                elif(prediction == 'en'):
                    si_en_uni += 1
#                     print (data['word'][i])
    except:
         continue

In [13]:
print(en_en_nn,en_si_nn,si_si_nn,si_en_nn)
print ("NN Model Evauation | Let English as Positive")
accuracy_nn  = (en_en_nn + si_si_nn )/(en_en_nn  + en_si_nn  + si_en_nn  + si_si_nn )
precision_nn  = en_en_nn /(en_en_nn  + en_si_nn )
recall_nn  = en_en_nn /(en_en_nn  + si_en_nn )
print ("Accuracy : ", accuracy_nn )
print ("Precision : ", precision_nn )
print ("Recall : ", recall_nn )

14635 4987 17673 3800
NN Model Evauation | Let English as Positive
Accuracy :  0.786178367197956
Precision :  0.7458464988278463
Recall :  0.7938703553024139


In [40]:
print ("Unicode Model Evauation | Let English as Positive")
accuracy_uni  = (en_en_uni + si_si_uni )/(en_en_uni  + en_si_uni  + si_en_uni  + si_si_uni )
precision_uni  = en_en_uni /(en_en_uni  + en_si_uni )
recall_uni  = en_en_uni /(en_en_uni  + si_en_uni )
print ("Accuracy : ", accuracy_uni )
print ("Precision : ", precision_uni)
print ("Recall : ", recall_uni )

Unicode Model Evauation | Let English as Positive
Accuracy :  0.9832836360350592
Precision :  0.9809733627077909
Recall :  0.9819566960705693
